# Sreality - All_In_One
## Jirka Zelenka
### 12.3.-24.4.2020
### Celý projekt = Scraping + Cleaning & Dropping + Vizualizace + All_In_One + PowerBI
-----------------------------------------------------------------------------

### Obsah:
* 1) Importování Packagů
* 2) Definice všech funkcí
* 3) Spuštění Procesu
* 4) Analýzy a grafy - TBD
* 5) Export do HTML a zaslání na mail  - TBD


-----------------------
# 1) Importování Packagů
-----------------------

In [ ]:

##### Obecné ############
import pandas as pd                     # for dataframes' manipulation
from pandas import DataFrame            # for creating dataframes
import numpy as np                      # for arrays
import matplotlib as plt                # for plotting
from matplotlib.pyplot import figure    # for saving and changing size of plots

from collections import Counter         # for counting elements 
from datetime import datetime           #for actual date
import re                               # !!! relativní Novinka - regular expressions
from time import sleep                  # for sleeping (slowing down) inside a function
import random                           # for random number (sleeping)
import math                             # Round float
import time                             # Time measuring
import itertools                        # for unlisting nested lists


##### Scraping ############
import requests                         # for robots check
from bs4 import BeautifulSoup           # for parsing
from selenium import webdriver          # for browsers control
import json                             # for Requests

##### GeoPy ############        
from geopy.geocoders import Nominatim   # Geolocator   # pip install geopy  
from geopy.exc import GeocoderTimedOut  # for Error handling

##########################
# Zaítm nepoužito:
##### Widgets ############
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
import os
from IPython.display import Image

##### Bonus - Hudba ############
import winsound                        # for Beep-sounds

##### Vizualizace ############
import seaborn as sns                  #for cool plots



import sys                             # ???

-----------------------
# 2) Definice všech funkcí
-----------------------

In [ ]:

################################################################################################################################
##############################################   SCRAPING    ###################################################################
################################################################################################################################

################################################################################################################################  1 = Scraping

def get_soup_elements(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):  
    
    browser = webdriver.Chrome()
    
    url_x = r"https://www.sreality.cz/hledani"             
    url = url_x + "/" +  typ_obchodu + "/" +  typ_stavby

    browser.get(url) 
    sleep(random.uniform(1.0, 1.5))
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,'lxml') # "parser" ??
    
    elements = []    
    
    for link in soup.findAll('a', attrs={'href': re.compile("^/detail/")}):     
        link = link.get('href')   
        elements.append(link)     
    elements = elements[0::2]   

    ##########################################
    # 3. zjištění počtu listů - mělo by být optional, ale nevadí
    ##########################################
    records = soup.find_all(class_ ='numero ng-binding')[1].text
    records = re.split(r'\D', str(records))                         
    records = ",".join(records).replace(",", "")
    records = int(records)
    max_page = math.ceil(records / 20)   
    print("----------------")
    print("Scrapuji: " + str(typ_obchodu) + " " + str(typ_stavby))
    print("Celkem inzerátů: " + str(records))
    print("Celkem stránek: " + str(max_page))
    
    ##########################################
    # 4. nastavení počtu stránek  -mělo být víc promakané
    ##########################################
    if pages == 999:      
        pages = max_page
    
    print("Scrapuji (pouze) " + str(pages) + " stran.")
    print("----------------")
    
    ##########################################
    # 4. Scrapping zbylých listů - naštěstí v jednom okně
    ##########################################    
    
    for i in range(pages-1):   
        i = i+2
        
        sys.stdout.write('\r'+ "Strana " + str(i-1) + " = " + str(round(100*(i-1)/(pages), 2)) + "% progress. Zbývá cca: " + str(round(random.uniform(3.4, 3.8)*(pages-(i-1)), 2 )) + " sekund.")    # Asi upravím čas, na rychlejším kabelu v obýváku je to občas i tak 3 sec :O

        url2 = url + "?strana=" + str(i)
        browser.get(url2)

        sleep(random.uniform(1.0, 1.5))

        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup2 = BeautifulSoup(innerHTML,'lxml') 
        
        elements2 = []
        
        for link in soup2.findAll('a', attrs={'href': re.compile("^/detail/prodej/")}):  
            link = link.get('href') 
            elements2.append(link)  
   
        elements2 = elements2[0::2]  
        
        elements = elements + elements2
    
    browser.quit()   
    
    return elements

################################################################################################################################  2 = Získání URLS
def elements_and_ids(x):
    
    elements = pd.DataFrame({"url":x})

    def get_id(x):
        x = x.split("/")[-1]
        return x

    elements["url_id"] = elements["url"].apply(get_id)
    
    len1 = len(elements)
    #Přidáno nově, v tuto chvíli odmažu duplikáty a jsem v pohodě a šetřím si čas dál.
    elements = elements.drop_duplicates(subset = [ "url", "url_id"], keep = "first", inplace = False)   
    len2 = len(elements)                                                                             
                                                                                                      
    print("-- Vymazáno " + str(len1-len2) + " záznamů kvůli duplikaci.")
    return elements

################################################################################################################################  3 = získání Souřadnic, Ceny a Popisu = z JSON
def get_coords_price_meters(x):
    
    url = "https://www.sreality.cz/api/cs/v2/estates/" + str(x)
    
    response = requests.get(url)
    byt = json.loads(response.text, encoding = 'UTF-8')                                
    try:
        coords = (byt["map"]["lat"], byt["map"]["lon"])
    except:
        coords = (0.01, 0.01)
    try:
        price = byt["price_czk"]["value_raw"] 
    except:
        price = -1
        
    try:
        description = byt["meta_description"]
    except:
        description = -1
    
    return coords, price, description


# Severní Šířka = latitude
# výchoDní / zápaDní Délka = longitude

def latitude(x):
    x = str(x).split()[0][1:8]
    return x

def longitude(x):
    x = str(x).split()[1][0:7]
    return x



################################################################################################################################  4 = Prodej + Dům + Pokoje = z URL
def characteristics(x):
    x = x.split("/")
    buy_rent = x[2]
    home_house = x[3]
    rooms = x[4]
    
    return buy_rent, home_house, rooms

################################################################################################################################  5 =  Plocha z Popisu- VYLEPŠENO

# Upravenopro čísla větší než 1000 aby je to vzalo
# Zároveň to už nebere Dispozice, Atpyický atd.

def plocha(x):
    try:
        metry = re.search(r'\s[12]\s\d{3}\s[m]', x)[0] # SPecificky popsáno: Začíná to mezerou, pak 1 nebo 2, pak mezera, pak tři čísla, mezera a pak "m"
        metry = metry.split()[0] + metry.split()[1]     # Separuju Jedničku + stovky metrů, bez "m"
    except:
        try:
            metry = re.search(r'\s\d{2,3}\s[m]', x)[0]  #Mezera, pak 1-3 čísla, mezera a metr
            metry = metry.split()[0]                    # Separuju čísla, bez "m"
        except:
            metry = -1
    return metry


################################################################################################################################  6 = Adresy z předešlých inzerátů a short_coords

def short_coords(x):
    """
    x = x.astype(str)   # Bylo potřeba udělat string - ale Tuple se blbě převádí - vyřešil jsem uložením a načtením skrz excel
    """
    
    x1 = re.split(r'\W+', x)[1] + "."+re.split(r'\W+', x)[2]
    x1 = round(float(x1), 4)

    x2 = re.split(r'\W+', x)[3] + "."+re.split(r'\W+', x)[4]
    x2 = round(float(x2), 4)

    return (x1, x2)

#############################

def adress_old(x):  # Napmapuje až 80 % Adres z předešlých inzerátů

    adresy = pd.read_excel("Adresy.xlsx")
    adresy = adresy[["oblast", "město", "okres", "kraj", "url_id", "short_coords"]]
    
    #Nejlepší napárování je toto:
    # porovnáno Inner a Left minus řádky s NaNs a funguje stejně)
    
    x.short_coords = x.short_coords.astype(str)   # Nově asi musím přidat toto - získat string na souřadnice, protože v Načteném adresáři je mám už taky jako string
    data = pd.merge(x, adresy, on=["short_coords", "url_id"], how = "left")  #upraveno matchování na url_ID + short_coords, je to tak iv Adresáří, je to jednoznačné, jsou tam unikátní. 
                                                                            # Pokud si v dalším kroku dostáhnu ke starému url_id a k nové coords ještě novou adresu, tak pak se mi uloží do Adresáře nová kombiance ID + short_coord a je to OK
                                                                             # Viz funkce"update_databáze_adres() kde je totéž info
            
    print("-- Počet doplněných řádků je: " + str(len(data[~data.kraj.isna()])) + ", počet chybějících řádků je: "   + str(len(data[data.kraj.isna()])))
    
    return data

################################################################################################################################ 7 = Adresy - zbývající přes GeoLocator

def adress_new(x):

# Pozn. - je to random, závislost rychlosti na user_agent, i na format_string se nepovedlo potvrdit - ale dokumentace user-agent uvíádí jako povinnost
# Naopak timeout na 20s a na None a zpátky mi nepomohl s rychlostí, ale minimálně zrušil Errory - None záleží na verzi geopy !! viz dokumentace
# Nejvíc podezřívám Wifi vs. kabel - 3 sec -> 1/3 sec, skoro vždy -> Ano, kabel v pokoji = 3-20s na záznam, kabel v obýváku = 0.25s na záznam !!!!!!!!!!!!!!!!§
# Problém s Too many requests se "spraví přes noc", kdyžtak - nebo viz stackoverflow - nastavit user-agent (https://stackoverflow.com/questions/22786068/how-to-avoid-http-error-429-too-many-requests-python)
  
    geolocator = Nominatim(timeout = 20, user_agent = "JZ_Sreality")   # Pomohlo změnit jméno, proti "Error 403" !!        
    location = geolocator.reverse(x.strip("())"))   
                                                    # Reverse samotné znamená obrácené vyhleádvání = souřadnice -> Adresa
     
    try:
        oblast  = location[0].split(",")[-7]
    except:
        oblast  = -1
    try:
        město = location[0].split(",")[-6]
    except:
        město  = -1
    try:
        okres = location[0].split(",")[-5]
    except:
        okres  = -1
    try:
        kraj = location[0].split(",")[-4]  
    except:
        kraj  = -1       
    
    time.sleep(0.5)
    return oblast, město, okres, kraj

##################################################################

# Není nutné, půjde i bez toho - když vypustím TIMEOUT - zatím ale nechávám
def repeat_adress(x):
    try:
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))
    except GeocoderTimedOut:
        print("Another try")
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))


################################################################################################################################ 8 = Merging adres

def adress_merging(x):

    data_new = x.copy()          
    bool_series = pd.isnull(data_new.kraj)                                   
    data_new = data_new[bool_series]     #subset s chybějícími adresami, už musím mít hotové ADRESS_OLD !!!!     
        
    # Musím takhle přímo použít na subset, data_new se tím updatuje
    # Ideální případ: cca 3561/3 = 1.200s = 20 min bez záseku -> vše chybějící
    repeat_adress(data_new)
        
    # Nakonec jsem zamítl .merge( all columns, how = "outer"), kdoví proč to dávalo 7 řádků navíc a tohle je čistší
    data_all = pd.concat([x, data_new], join_axes=[x.columns])   
    data_all = data_all[~pd.isnull(data_all.kraj)]
    data_all = data_all.sort_index()

    data = data_all.copy()
    
    return data

################################################################################################################################
##############################################   CLEANING    ###################################################################
################################################################################################################################

################################################################################################################################ 1 = Mezery u Adres

############ Mezery před názvy (všeho) a pomlčky doplněné z GeoPy
def smaž_mezery(x):
    x = re.findall(r'\w+', str(x))   # Když napíšu tento regex (r'\w+-?\w+'), tak bych i zachoval "pomlčku", 
    x = ' '.join(x)                  # ale to já naopak nechci, vyřeším si dvě věci najednou 
    
    return x

################################################################################################################################ 2 = Posunutý Okres, Kraj...

def kraj_check(x):
    x = x.split()[0]
    return x


def uprav_adresy(x):

    x["kraj_check"] = x["kraj"].apply(kraj_check)

    # U těch řádků, kde byl okres v názvu kraje posuň všechny tři hodnoty "doleva"
    x.oblast[x["kraj_check"] == "okres"] = x.město[x["kraj_check"] == "okres"].copy()
    x.město[x["kraj_check"] == "okres"] = x.okres[x["kraj_check"] == "okres"].copy()
    x.okres[x["kraj_check"] == "okres"] = x.kraj[x["kraj_check"] == "okres"].copy()
    x.kraj[x["kraj_check"] == "okres"] = -1

    # Oříznutý dataset OBCE
    obce = pd.read_excel(r"obce_okresy_kraje.xlsx")
    obce_kus = obce[["okres", "kraj"]].copy()

    # LEFT Merge by měl přiřadit okresům správné Kraje z df obcí
    data = pd.merge(x, obce_kus, on=['okres'], how = "left")

    # Smazání případných duplikátů
    data.drop_duplicates(keep = "first", inplace = True)

    # Přiřazení nového sloupečku na starý
    data.kraj_x[data["kraj_check"] == "okres"] = data.kraj_y[data["kraj_check"] == "okres"].copy()

    # Přejmenování a získání lepších sloupců
    data = data.rename(columns={'kraj_x': 'kraj'})
    data = data[["popis", "prodej", "dům", "pokoje", "plocha", "oblast", "město", "okres", "kraj", "cena", "url", "url_id", "coords", "short_coords", "lat", "lon"]]  #Doplněno LAT a LON nově

    data.reset_index(inplace = True, drop = True)

    return data
################################################################################################################################ 3 = Počet obyvatel z Excelu ("obce_okresy_kraje.xlsx")

def počet_obyvatel(x):

    obce = pd.read_excel("obce_okresy_kraje.xlsx")
    obce_kus = obce[["město", "okres", "kraj", "obyvatelé"]].copy()
    data = pd.merge(x, obce_kus ,on = ["město", "okres", "kraj"], how = "left")
    data.drop_duplicates(keep = "first", inplace = True)

    return data

################################################################################################################################ 4 = Update databáze adres

def update_databáze_adres(x):
    
    adresy = pd.read_excel("Adresy.xlsx")
    len1 = len(adresy)
    print("-- Původní délka adresáře: " + str(len1))

    #Oříznu si data na to, co chci uložit a přidám aktuální datum
    data2 = x[["oblast", "město", "okres", "kraj", "url", "url_id", "short_coords"]]  # Odebráno Coords, vznikaly duplikáty zbytečně
    data2["Datum"] = datetime.datetime.now().strftime("%Y-%m-%d")

    # Připojím nová data, dropnu duplikáty (mažu odzadu), resetnu indexy, změřím délku
    # Přírůstek by měl odpovídat počtu řádků dohledaných přes Geopy minus duplikáty případně
    adresy = pd.concat([adresy, data2])
    adresy = adresy.drop_duplicates(subset = [ "url_id", "short_coords"], keep = "first", inplace = False)
    
    ### Nově vloženo: Odebrat shitty adresy - protože jinak jsem -1 kraj dropoval až ve fázi 3  !!!!!!!!!!!!!!!!!!!!!!!!!!
    adresy = adresy[adresy.short_coords != "(0.01, 0.01)"]
    
    # Změněna logika, dropni všechno kde máš dvojí varianty pro kombinaci url_id + short_coords, protože přesně tohohle se to mapuje a dělá to pak rozdvojení různá. !!
    # Jinak ponechává to možnost že mám jedno ID vícekrát s různými short_coords, ale to je možné, nebrání to zpětnému mapování
    adresy.reset_index(inplace = True, drop = True)
    len2 = len(adresy)
    print( "-- Nová délka adresáře: " + str(len2) + ", přidáno "+ str(len2-len1) + " záznamů. Ukládám do Adresy.xlsx")

    return adresy


################################################################################################################################ 5 = Cena na tisíce

# Zatím vypnu 
"""
def cena_tisíce(y):

    y["cena"] = y["cena"].apply(lambda x: int(round(x/1000, 0)))

    return y
"""
################################################################################################################################ 6 = Cena za metr

def cena_metr(y):

    y["cena_metr"] = y["cena"].astype(int)/y["plocha"].astype(int)
    y["cena_metr"] = y["cena_metr"].apply(lambda x: round(x, 1))
    
    return y

################################################################################################################################
##############################################   DROPPING    ###################################################################
################################################################################################################################

################################################################################################################################ 1 = Missing values

def missing_values(x):
    
    data = x.copy()
    
    print("-- Celkem záznamů bez ceny: "+ str( len(data[data["cena"] == 0]) )   )
    print("-- Celkem záznamů bez popisu => plochy: "+ str( len(data[data["plocha"] == -1]) ) )
    print("-- Celkem záznamů bez okresu a kraje (pouze starší data): "+ str(len(data[data["kraj"] == "1"]))) 
    # Pokoje by měly být OK
    # Obyvatelé taky
    # Cena_metr pak už taky
    
    data = data[data.cena != 0]
    data = data[data.plocha != -1]
    data = data[data.kraj != "1"]
    
    
    return data

################################################################################################################################ 2 = Extrémy

def extrémy(x):
    
    data = x.copy()     # Upraveno aby to korespondovalo s Tisíci
    
    data = data[data.cena > 50000] # Zatím nikdy nebyl byt pod 100.000, resp za 90.000 půl byt
    data = data[data.plocha > 10] # Zatím nikdy nebyla plocha pod 12, snad je to rozumný předpoklad
    data = data[data.cena_metr < 450000]   # Zatím nikdy nebyla Víc než 360 za metr, jen v jednom chybném zadání rovnou 568 !!
    data = data[data.cena_metr > 1000]   #Toto se občas vidí a je to očividná chyba v zadání inzerátu. Viz CHlumec - 376m !! 338.000

    return data


################################################################################################################################ 3 = Duplikáty

def duplikáty(x):
    
    data =  x.copy()
    a = len(data[data.duplicated([ "popis", "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "obyvatelé", "cena", "cena_metr", "coords", "short_coords", "lat", "lon"],keep= "first")]) # Jen různá URL a ID
    #display(len(data[data.duplicated([ "popis", "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "obyvatelé", "cena", "cena_metr", ],keep= "first")]))                  # Ještě různé souřadnice
    #display(len(data[data.duplicated([ "popis", "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "obyvatelé", "coords", "short_coords"  ],keep= "first")]))          # Různé ceny a ceny za metr
    #display(len(data[data.duplicated([ "popis", "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "obyvatelé" ],keep= "first")]))                                       #Různé souřadcnie a ještě různé ceny
                                                                                                                                                                                        # Tím že povolím různé ceny duplikáty nepřibydou
    print("-- Počet řádků, které k sobě mají duplikáty (krom url a url_id): " + str(a))                                                                                                               #Ty bych asi byl ochoten tolerovat, narozdíl od těch 215. -> Zůstanu u varianty a)
    data = data.drop_duplicates(subset =[ "popis", "prodej", "dům", "pokoje", "plocha","oblast", "město", "okres", "kraj", "obyvatelé", "cena", "cena_metr", "coords", "short_coords", "lat", "lon" ],keep= "first", inplace = False) 
    #Z dat dropnu duplikáty podle všeho krom url a url_id, nechávám si první záznamy
    
    return data



In [ ]:

################################################################################################################################
##############################################   SCRAPING    ###################################################################
################################################################################################################################

def scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):
    
    # Scrapni data - hezky komunikuje = cca 50 min
    data = get_soup_elements(typ_obchodu = typ_obchodu, typ_stavby = typ_stavby, pages = pages)
    print( "1/8 Data scrapnuta, získávám URLs.")
    
    # 2 = Získání URLS
    data = elements_and_ids(data)
    data.to_excel(r"a1_URLs_prodej_byty.xlsx")
    print( "2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...")
    
    
    # 3 = získání Souřadnic, Ceny a Popisu = z JSON
    data["coords"], data["cena"], data["popis"] = zip(*data['url_id'].map(get_coords_price_meters))
    data["lat"] = data["coords"].apply(latitude)
    data["lon"] = data["coords"].apply(longitude)
    data.to_excel(r"a2_Souřadnice_prodej_byty.xlsx")
    print( "3/8 Získány Souřadnice, Ceny a Popis, nyní získávám informace z URLs.")
   
    # 4 = Prodej + Dům + Pokoje = z URL
    data["prodej"], data["dům"],  data["pokoje"] = zip(*data['url'].map(characteristics))
    print("4/8 Získány informace z URLs, nyní získávám informace z popisu.")
    
    # 5 =  Plocha z Popisu
    data["plocha"] = data['popis'].apply(plocha)
    data.to_excel(r"a3_Popisky_prodej_byty.xlsx")
    print( "5/8 Získány informace z Popisu, nyní mapuji Adresy z předešlých inzerátů.")
    
   
    # 6 = Adresy z předešlých inzerátů a short_coords
    data = pd.read_excel(r"a3_Popisky_prodej_byty.xlsx")   # Abych se vyhnul konverzi TUPLE na STRING, což není triviální, tak si to radši uložím a znova načtu a získám stringy rovnou. Snad mi to nerozbije zbytek
    data["short_coords"] = data["coords"].apply(short_coords)
    data_upd = adress_old(data)                                # Tady nepotřebuji maping, protože se nesnažím něco nahodit na všechny řádky, ale merguju celé datasety
    data = data_upd.copy()
    print( "6/8 Namapovány Adresy z předešlých inzerátů, nyní stahuji nové Adresy - několik minut...")            # Přidat do printu počet řádků, kolik mám a kolik zbývá v 7. kroku

    # 7-8 = Adresy - zbývající přes GeoLocator + Merging

    try:                                    # !!! Riskuju že zas něco selže, jako USER- AGENT posledně...
        data_upd = adress_merging(data)    #Přidáno TRY pro situace, kdy už mám všechyn adresy z OLD a nejde nic namapovat !
        data = data_upd.copy()
        data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
        print("7+8/8 Získány nové adresy + mergnuto dohromady. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    
    except:
        data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
        print("7+8/8 ŽÁDNÉ nové adresy. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    

    return data


In [ ]:

################################################################################################################################
##############################################   CLEANING    ###################################################################
################################################################################################################################

def clean_all(x):
    
    #data = pd.read_excel(x)   # Pracovní verze s načítáním dat, jinak napojeno na Scrap
    data = x.copy()
    
    print("----------------")
    print("Počet záznamů před čištěním: " + str(len(data)))
    
    
    # 1 = Mezery u adres 
    data["oblast"] = data["oblast"].apply(smaž_mezery)
    data["město"] = data["město"].apply(smaž_mezery)
    data["okres"] = data["okres"].apply(smaž_mezery)
    data["kraj"] = data["kraj"].apply(smaž_mezery)
    print("1/6 Vymazány mezery před názvy, následuje posouvání rozbitých okresů a krajů")
    
    # 2 = Posunutý Okres, Kraj...
    data = uprav_adresy(data)
    print("2/6 Posunuty rozbité okresy a kraje, následuje doplnění počtu obyvatel.")
     
    # 3 = Počet obyvatel z Excelu
    data = počet_obyvatel(data)
    data.to_excel("a5_Adresy_a_obyvatele_data_prodej_byty.xlsx")
    print("-- Aktuální délka datasetu: " + str(len(data)))
    print("3/6 Doplněn počet obyvatel, updatuji databázi adres.")
    
    # 4 = Update databáze adres 
    adresy = update_databáze_adres(data)
    adresy.to_excel(r"Adresy.xlsx")
    print("4/6 Databáze Adres updatována. Délka datasetu: " + str(len(data)) + " záznamů. Převádím ceny na tisíce")    

    # 5 Cena na tisíce - zatím vypínám kvůli POWER BI
    #data = cena_tisíce(data)
    print("5/6 Ceny NECHCI ABY BYLY převedeny na tisíce. Počítám ceny za metr.")
    
    # 6 Cena za metr
    data = cena_metr(data)
    data["datum"] = datetime.datetime.now().strftime("%d.%m.%Y")   # Nově přidáno kvůli souhrnným datům
    data = data[["popis", "prodej", "dům", "pokoje", "plocha", "oblast", "město", "okres", "kraj", "obyvatelé", "cena", "cena_metr", "url", "url_id", "coords", "short_coords", "lat", "lon", "datum"]]
    data.to_excel("a6_CLEANED_data_prodej_byty.xlsx")
    print("6/6 Spočítány ceny za metr. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 2.")
    
          
    return data


In [ ]:

################################################################################################################################
##############################################   DROPPING    ###################################################################
################################################################################################################################

def drop_all(x):
    
    #data = pd.read_excel(x)   # Pracovní verze s načítáním dat, jinak napojeno na Cleaning
    data = x.copy()
    x = len(data)
    print("----------------")
    print("Počet záznamů před čištěním: " + str(x))
    
    
    # 1 = Missing Values
    data = missing_values(data)
    y = len(data)
    print("1/3 Vymazány chybějící hodnoty (Ceny, Plochy, (Kraje)), celkem " + str(x-y) + " řádků. Zbývá " + str(y) + " záznamů.")

    # 2 = Extrémy
    data = extrémy(data)
    z = len(data)
    print("2/3 Vymazány extrémní hodnoty (Ceny, Plochy, Cena za metr, celkem " + str(y-z) + " řádků. Zbývá " + str(z) + " záznamů.")
    
    # 3 = Duplikáty
    data = duplikáty(data)
    b = len(data)
    data.to_excel("a7_DROPPED_data_prodej_byty.xlsx")
    print("3/3 Vymazány duplikáty, zbývá " + str(b) + " záznamů. Konec Fáze 3.")
    
    display(data.head())
    
    return data

-----------------------
# 3) Spuštění Procesu
-----------------------

In [ ]:
### Nastaveno na 2 zkušební stránky
# pages = 999 spustí všechny stránky

t1 = time.time()

data = drop_all(clean_all( scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", pages = 2)))

t2 = time.time()
                                     
data_all = pd.read_excel("data_prodej_byty_souhrn.xlsx")
display(len(data_all))
                                    
data_new = pd.concat([data_all, data])
display(len(data_new))

data_new.to_excel("data_prodej_byty_souhrn_NEW.xlsx")

t3 = time.time()

print("První 3 fáze: " + str(t2-t1))
print("Souhrnná data: " + str(t3-t2))


In [ ]:
data.nunique() 

In [ ]:
data.isin([-1]).any()

-----------------------
# 4) Analýzy a grafy - TBD
-----------------------

-----------------------
# 5) Export do HTML a zaslání na mail  - TBD
-----------------------